In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
### Read in values from text file
file_path = r"Cc_Input.txt"  # Replace with the actual path to your file

values = {}
with open(file_path, "r") as file:
    line = file.readline().strip()
    while line:
        if line.startswith("$$"):
            key = line[2:].strip()
            value = float(file.readline().strip())
            values[key] = value
        line = file.readline().strip()

In [4]:
### Inputs
Pre_ConsolStress = values["PRECONSOLIDATION_STRESS"]
Cc = values["Cc"] # Virgin compression slope
Cr_ratio = values["Cr/Cc"] # Ratio of C_r to C_c
e0 = values["e0"] # Initial void ratio
ef = values["ef"] # Final value of void ratio
input_num_points = int(values["Input_Num_Points"]) # Number of spaces avalialable for inputs in PSDDF


In [5]:
num_points = input_num_points + 1 # Add a data point so that when the Cr and Cc arrays are later appened the length is correct

In [6]:
# Calculate the maximum eff_stres possible before e becomes negative
max_stress = 10**((e0-ef)/Cc + np.log10(Pre_ConsolStress) * (-Cr_ratio + 1))

# Percentage of the loading that is reloading (swell)
# (Used to calculate the number of points that should be used for the recompression)
Perc_reloading = (Pre_ConsolStress-1)/max_stress

In [7]:
# Number of points to generate
# num_points = 100 + 1

# Number of recompression points
num_Cr_points = int(num_points * Perc_reloading + 1)
if num_Cr_points < 4:
    num_Cr_points = 4

# Number of virgin compression points
num_Cc_points = num_points-num_Cr_points

In [8]:
### Define arrays of effective stress points

#### Virgin Compression initialization
Cc_eff_stress = np.linspace(Pre_ConsolStress, max_stress, num_Cc_points)

#### Recompression initializations 
Cr_eff_stress = np.linspace(1, Pre_ConsolStress,num_Cr_points)

In [9]:
### Void ratio points for recompression
e_recomp = -1 * Cr_ratio * Cc * np.log10(Cr_eff_stress) + e0

### Void ratio for virgin compression

# Calculate intercept for Virgin compression line
b = e_recomp[-1] + Cc * np.log10(Pre_ConsolStress)

# Calulcate void ratios for virgin compression line
e_virgin = -Cc * np.log10(Cc_eff_stress) + b

### Append the two arrays
eff_stress = np.append(Cr_eff_stress, Cc_eff_stress[1:])
es = np.append(e_recomp, e_virgin[1:])